## Config

In [1]:
import os

# Force set environment variables (this is essential for Jupyter notebooks)
os.environ['NUPLAN_MAPS_ROOT'] = '/mnt/hdd8/navsim_dataset/maps'
os.environ['NUPLAN_MAP_VERSION'] = 'nuplan-maps-v1.0'
os.environ['OPENSCENE_DATA_ROOT'] = '/mnt/hdd8/navsim_dataset'

print("\nAfter setting:")
print(f"NUPLAN_MAPS_ROOT: {os.getenv('NUPLAN_MAPS_ROOT')}")
print(f"NUPLAN_MAP_VERSION: {os.getenv('NUPLAN_MAP_VERSION')}")
print(f"OPENSCENE_DATA_ROOT: {os.getenv('OPENSCENE_DATA_ROOT')}")


After setting:
NUPLAN_MAPS_ROOT: /mnt/hdd8/navsim_dataset/maps
NUPLAN_MAP_VERSION: nuplan-maps-v1.0
OPENSCENE_DATA_ROOT: /mnt/hdd8/navsim_dataset


In [ ]:
from pathlib import Path
import hydra
from hydra.utils import instantiate
from navsim.common.dataloader import SceneLoader
from navsim.common.dataclasses import SceneFilter, SensorConfig

SPLIT = "mini"  # ["mini", "test", "trainval"]
FILTER = "all_scenes"

hydra.initialize(
    config_path="../navsim/planning/script/config/common/train_test_split/scene_filter"
)
cfg = hydra.compose(config_name=FILTER)
scene_filter: SceneFilter = instantiate(cfg)
openscene_data_root = Path(os.getenv("OPENSCENE_DATA_ROOT"))

scene_loader = SceneLoader(
    openscene_data_root / f"navsim_logs/{SPLIT}", # data_path
    openscene_data_root / f"sensor_blobs/{SPLIT}", # original_sensor_path
    scene_filter,
    openscene_data_root / "warmup_two_stage/sensor_blobs", # synthetic_sensor_path
    openscene_data_root / "warmup_two_stage/synthetic_scene_pickles", # synthetic_scenes_path
    sensor_config=SensorConfig.build_all_sensors(),
)

/tmp/ipykernel_865950/2741946232.py:12: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(
Loading logs: 100%|██████████| 64/64 [00:32<00:00,  1.94it/s]


## Print PDM with single token

In [ ]:
# Select token from metric cache
# You can modify the path to any cache

import numpy as np
from pathlib import Path
from navsim.common.dataloader import MetricCacheLoader

workspace_cache = Path("/mnt/hdd5/Qiaoceng/navsim_workspace/exp/metric_cache")
cache_loader = MetricCacheLoader(workspace_cache)
available_tokens = cache_loader.tokens

token = np.random.choice(available_tokens)
scene = scene_loader.get_scene_from_token(token)
print(f"🎯 Selected token from cache: {token}")

metric_cache = cache_loader.get_from_token(token)
print("✅ Successfully loaded scene and metric cache!")

In [ ]:
# Plot BEV with agent

# from navsim.agents.constant_velocity_agent import ConstantVelocityAgent
from navsim.agents.transfuser.transfuser_agent import TransfuserAgent
from navsim.agents.transfuser.transfuser_config import TransfuserConfig
from navsim.visualization.plots import plot_bev_with_agent
import matplotlib.pyplot as plt

# agent = ConstantVelocityAgent()
config = TransfuserConfig()
config.checkpoint_path = "../model/LatentTransfuser_mini/0830.ckpt"
agent = TransfuserAgent(config, scene)
fig, ax = plot_bev_with_agent(scene, agent)
plt.title(f"Scene token: {token}")
plt.show()

In [ ]:
# Compute PDM score

from navsim.evaluate.pdm_score import pdm_score
from navsim.planning.simulation.planner.pdm_planner.simulation.pdm_simulator import PDMSimulator
from navsim.planning.simulation.planner.pdm_planner.scoring.pdm_scorer import PDMScorer, PDMScorerConfig
from navsim.traffic_agents_policies.navsim_IDM_traffic_agents import NavsimIDMTrafficAgents
from navsim.planning.simulation.observation.navsim_idm_agents import NavsimIDMAgents
from nuplan.planning.simulation.trajectory.trajectory_sampling import TrajectorySampling

print("🚀 Setting up PDM score components...")
agent_input = scene.get_agent_input()
if agent.requires_scene:
    trajectory = agent.compute_trajectory(agent_input, scene)
else:
    trajectory = agent.compute_trajectory(agent_input)

proposal_sampling = TrajectorySampling(num_poses=40, interval_length=0.1)
simulator = PDMSimulator(proposal_sampling=proposal_sampling)

scorer_config = PDMScorerConfig(
    progress_weight=5.0,
    ttc_weight=5.0,
    lane_keeping_weight=2.0,
    history_comfort_weight=2.0,
    two_frame_extended_comfort_weight=2.0,
    driving_direction_horizon=1.0,
    driving_direction_compliance_threshold=2.0,
    driving_direction_violation_threshold=6.0,
    stopped_speed_threshold=5e-03,
    future_collision_horizon_window=1.0,
    progress_distance_threshold=5.0,
    lane_keeping_deviation_limit=0.5,
    lane_keeping_horizon_window=2.0,
    human_penalty_filter=True
)
scorer = PDMScorer(proposal_sampling=proposal_sampling, config=scorer_config)

idm_agents = NavsimIDMAgents(
    target_velocity=10,
    min_gap_to_lead_agent=1.0,
    headway_time=1.5,
    accel_max=1.0,
    decel_max=2.0,
    open_loop_detections_types=[],
    minimum_path_length=20,
    planned_trajectory_samples=None,
    planned_trajectory_sample_interval=None,
    radius=100,
    add_open_loop_parked_vehicles=True,
    idm_snap_threshold=3.0
)
traffic_agents_policy = NavsimIDMTrafficAgents(
    idm_agents_observation=idm_agents,
    future_trajectory_sampling=simulator.proposal_sampling
)

print("📊 Computing PDM score...")
score_result = pdm_score(
    metric_cache=metric_cache,
    model_trajectory=trajectory,
    future_sampling=simulator.proposal_sampling,
    simulator=simulator,
    scorer=scorer,
    traffic_agents_policy=traffic_agents_policy
)

print("✅ PDM score computation done!")

In [ ]:
# Add pdm score on the BEV plot

fig, ax = plot_bev_with_agent(scene, agent)
plt.subplots_adjust(top = 0.8)  # Adjust top to make space for the text
plt.title(f"Scene token: {token}")

if isinstance(score_result, tuple) and len(score_result) == 2:
    score_df, _ = score_result
plt.figtext(0.5, 0.855,
           f"Ego Progress: {score_df['ego_progress'].iloc[0]:.3f}", 
           ha='center', fontsize=8)

plt.show()

## Check Agent Load

In [ ]:
# 在現有代碼的最後添加：
try:
    if 'agent' in globals():
        print(f"🤖 Agent 檢查:")
        print(f"   Agent 類型: {type(agent).__name__}")
        if hasattr(agent, '_transfuser_model'):
            model_loaded = agent._transfuser_model is not None
            print(f"   模型已載入: {model_loaded}")
            if hasattr(agent, 'config') and hasattr(agent.config, 'checkpoint_path'):
                print(f"   模型路徑: {agent.config.checkpoint_path}")
        else:
            print(f"   Agent 沒有 _transfuser_model 屬性")
    else:
        print("⚠️ Agent 尚未創建")
except Exception as e:
    print(f"❌ Agent 檢查失敗: {e}")

In [ ]:
# 檢查模型文件
model_path = Path("../model/LatentTransfuser_mini/0830.ckpt")
print(f"📁 模型文件檢查:")
print(f"   路徑: {model_path}")
print(f"   存在: {model_path.exists()}")
if model_path.exists():
    size_mb = model_path.stat().st_size / (1024*1024)
    print(f"   大小: {size_mb:.1f} MB")

In [ ]:
import torch
try:
    model_data = torch.load("../model/LatentTransfuser_mini/0830.ckpt", map_location='cpu')
    print("✅ 模型文件可以被 PyTorch 載入")
    print(f"   模型keys: {list(model_data.keys())}")
except Exception as e:
    print(f"❌ 模型文件載入失敗: {e}")

In [ ]:
# 調試 TransfuserAgent 模型載入
try:
    # 重新創建一個 agent，但捕獲所有輸出
    print("🔧 調試 TransfuserAgent 載入過程...")
    
    test_config = TransfuserConfig()
    test_config.checkpoint_path = "../model/LatentTransfuser_mini/0830.ckpt"
    
    # 檢查 TransfuserAgent 的 __init__ 方法
    print("📝 開始創建 TransfuserAgent...")
    test_agent = TransfuserAgent(test_config, scene)
    
    # 檢查 agent 的所有屬性
    print(f"🔍 Agent 屬性檢查:")
    for attr in dir(test_agent):
        if not attr.startswith('_'):
            continue
        if 'model' in attr.lower():
            value = getattr(test_agent, attr, None)
            print(f"   {attr}: {type(value)} = {value}")
    
    # 嘗試手動載入到 agent
    print(f"🛠️ 嘗試手動設置模型...")
    model_data = torch.load("../model/LatentTransfuser_mini/0830.ckpt", map_location='cpu')
    print(f"   State dict keys: {len(model_data['state_dict'])} 個參數")
    
except Exception as e:
    print(f"❌ 調試失敗: {e}")
    import traceback
    traceback.print_exc()